In [7]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Model

# Define a CNN-based autoencoder
def create_cnn_autoencoder():
    input_layer = Input(shape=(1000, 6, 1))  # Adjust the input shape

    # Encoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    # ... add more layers as needed

    # Decoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    # ... add more layers as needed

    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_layer, decoded)
    return autoencoder

DIR = "../data/Raw/logRaws/"

NUM_SAMPLES = 2662
NUM_FEATURES = 6
NUM_FRAMES = 1000

file_names = os.listdir(DIR)
input_array = np.empty((NUM_SAMPLES, NUM_FRAMES, NUM_FEATURES))

# prepare data
def read_files(DIR):
    all_files = os.listdir(DIR)

    input_data = np.empty((0, 6), float)

    for file in all_files:
        file_path = os.path.join(DIR, file)
        df = pd.read_csv(file_path, usecols=[1, 19, 20, 21, 25, 26, 27], header=None)
        if df.shape[0] < 999:
            continue

        df.columns = ['frame', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
        df_input = df[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
        df_input = df_input.astype('float32')

        input_data = np.append(input_data, df_input, axis=0)
    input_data = input_data.reshape(NUM_SAMPLES, NUM_FRAMES, NUM_FEATURES)

    return input_data

input_data = read_files(DIR)

print(input_data.shape)


In [4]:
# Flatten data
data_flattened = input_data.reshape(input_data.shape[0], -1)

# split data
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(data_flattened, test_size=0.2, random_state=42)

In [5]:
# Create and compile the CNN autoencoder
autoencoder = create_cnn_autoencoder()
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))


2023-11-22 17:17:58.126810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 17:17:58.392591: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-11-22 17:17:58.767774: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51096000 exceeds 10% of free system memory.


Epoch 1/50


2023-11-22 17:17:59.083590: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51096000 exceeds 10% of free system memory.


ValueError: in user code:

    File "/home/vietman2/Workspace/.venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/vietman2/Workspace/.venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/vietman2/Workspace/.venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/vietman2/Workspace/.venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/vietman2/Workspace/.venv/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/vietman2/Workspace/.venv/lib/python3.9/site-packages/kera